<a href="https://colab.research.google.com/github/elizabethts/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp, ttest_ind

In [17]:
voting_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

#define column headers
column_headers = ('party','handicapped-infants', 'water-project-cost-sharing'
               , 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid'
               , 'religious-groups-in-schools', 'anti-satellite-test-ban'
               , 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration'
               , 'synfuels-corporation-cutback', 'education-spending'
               , 'superfund-right-to-sue', 'crime', 'duty-free-exports'
               , 'export-administration-act-south-africa')

#create dataframe
df = pd.read_csv(voting_url, names=column_headers)

#clean values
yes_no = {'y': 1, 'n': 0, '?': np.nan}
df = df.replace(yes_no)

df.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [18]:
df.describe()

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
df.isnull().sum()

party                                       0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64

##Hypothesis testing

In [0]:
#create subsets of each party
rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

In [24]:
print('Democratic Support: ', dem['immigration'].mean())
print('Repuplican Support: ', rep['immigration'].mean())

Democratic Support:  0.4714828897338403
Repuplican Support:  0.5575757575757576


In [29]:
ttest_ind(dem['immigration'],rep['immigration'], nan_policy = 'omit')

Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

#### The null hypothesis is that Democrats and Republicans do not support immigration equally
#### Alternative hypothesis: Democrats and republicans have equal support for immigration
#### Conclusion: Democrats and Republicans have equal support for immigration (p<0.1)
#### I REJECT the null hypotheis that democrats and republicans do not support immigration equally

In [40]:
print('Democratic Support: ', dem['anti-satellite-test-ban'].mean())
print('Repuplican Support: ', rep['anti-satellite-test-ban'].mean())

Democratic Support:  0.7722007722007722
Repuplican Support:  0.24074074074074073


In [41]:
ttest_ind(dem['anti-satellite-test-ban'],rep['anti-satellite-test-ban'], nan_policy = 'omit')

Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

#### The null hypothesis is that Democrats and Republicans support anti-satellite test ban at the same rate
#### Alternative hypothesis: Democrats support anti-satellite test ban at a greater rate than democrats
#### Democrats have a higher support for anti-satellite test ban than republicans (p<0.01)
#### I REJECT the null hypotheis that there is the same level of support for this bill among democrats and republicans

In [37]:
print('Democratic Support: ', dem['physician-fee-freeze'].mean())
print('Repuplican Support: ', rep['physician-fee-freeze'].mean())


Democratic Support:  0.05405405405405406
Repuplican Support:  0.9878787878787879


In [39]:
ttest_ind(dem['physician-fee-freeze'],rep['physician-fee-freeze'], nan_policy = 'omit')

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)

#### The null hypothesis is that Democrats and Republicans support physician fee freeze at the same rate
#### Alternative hypothesis: Republicans support physician fee freeze at a greater rate than democrats
#### Republican's have a higher support for physician fee freeze than democrats (p<0.01)
#### I REJECT the null hypotheis that there is the same level of support for this bill among democrats and republicans
